# CA1

# BFS

In [44]:

import numpy as np
import pandas as pd
import math as Math
import time
import itertools
import copy


        
class Cell:
    def __init__(self,empty=True,castle=False,Orc=False,watched=False
               ,watched_orcN=-1,watched_orcID=-1,Fello_pos=False,Fello_ID=-1,home=False,home_ID=-1):
        self.empty = empty
        self.castle = castle
        self.Orc = Orc
        self.watched = watched
        self.watched_orcN = watched_orcN
        self.watched_orcID = watched_orcID
        self.Fello_pos = Fello_pos
        self.Fello_ID = Fello_ID
        self.home = home
        self.home_ID = home_ID
       
    
class Gandalf:
    def __init__(self,row,col,path='',count = 0,alone = True, Fello_ID = -1):
        self.row = row
        self.col = col
        self.path = path
        self.count = count
        self.alone = alone
        self.Fello_ID = Fello_ID
    def __eq__(self, other):
        return self.count == other.count
        
        
class Board:
    def __init__(self,rows,cols,gandalf,board,pieces,parent=None):
        self.rows = rows
        self.cols = cols
        self.board = board
        self.gandalf = gandalf
        self.pieces = pieces
        self.parent = parent
        
    def __eq__(self, other):
        return self.pieces == other.pieces and self.gandalf==other.gandalf
       
    def __hash__(self):
        return hash((tuple(self.pieces)))

 
def show_path(finalState,initialState):
    
    if(finalState == initialState):
        show(finalState)
        return
    show_path(finalState.parent,initialState)
    show(finalState)
    
def checkEnd(state):
    
    if not state.board[(state.gandalf.row,state.gandalf.col)].castle:
        return False
    
    for i in range(1,len(state.pieces)):
        if(state.pieces[i]==(-1,-1)):
            return False
        if state.board[state.pieces[i]].home == True:
            if (state.board[state.pieces[i]].Fello_ID != state.board[state.pieces[i]].home_ID):
                    return False
        else:
            return False
    
    return True
            
                
                
    
def BFSSearch(initialState):
    numOfVisitedStates = 0
    visited = set()
    frontier = []
    frontier_set = set()
    frontier.append(initialState)
    frontier_set.add(initialState)
    while len(frontier) != 0:
       
        currentState = frontier.pop(0)
        visited.add(currentState)
        numOfVisitedStates += 1
        if(checkEnd(currentState)):
            return numOfVisitedStates,currentState


        moves={'U':(-1,0),'D':(1,0),'R':(0,1), 'L':(0,-1)}
        
        x,y = currentState.gandalf.row, currentState.gandalf.col
        for move in moves: 
            if(valid(x,y,x+moves[move][0], y+moves[move][1],currentState) == False):     
                continue
            newState = get_new_state(x+moves[move][0], y+moves[move][1],x,y,currentState,move)
            if (newState not in visited) and (newState not in frontier_set):
                newState.parent = currentState
                frontier.append(newState)
                frontier_set.add(newState)
                
    return -1,False
 
   
def move_to_empty_cell(new_x,new_y,curr_x,curr_y,currentState,move):
    newSateGandalf = copy.copy(currentState.gandalf)
    newSateGandalf.row, newSateGandalf.col = new_x,new_y
    newSateGandalf.path += move
    newSateGandalf.count = 0
    updated_pieces = copy.copy(currentState.pieces)
    updated_pieces[0] = (new_x,new_y)
    newState = Board(currentState.rows,currentState.cols,newSateGandalf,currentState.board, updated_pieces)
    return newState

def move_to_watched_cell(new_x,new_y,curr_x,curr_y,currentState,move):
    newSateGandalf = copy.copy(currentState.gandalf)
    newSateGandalf.row, newSateGandalf.col = new_x,new_y
    newSateGandalf.path += move
    updated_pieces = copy.copy(currentState.pieces)
    updated_pieces[0] = (new_x,new_y)
    if(currentState.board[new_x,new_y].watched_orcID != currentState.board[curr_x,curr_y].watched_orcID):
        newSateGandalf.count = 1
        newState = Board(currentState.rows,currentState.cols,newSateGandalf,currentState.board, updated_pieces)
        return newState
    if(currentState.gandalf.count != currentState.board[new_x,new_y].watched_orcN):
        newSateGandalf.count += 1
        newState = Board(currentState.rows,currentState.cols,newSateGandalf,currentState.board, updated_pieces)
        return newState
       
def move_to_home_cell(new_x,new_y,curr_x,curr_y,currentState,move):
    newSateGandalf = copy.copy(currentState.gandalf)
    newSateGandalf.row, newSateGandalf.col = new_x,new_y
    newSateGandalf.path += move
    newSateGandalf.count = 0
    newSateGandalf.alone = True
    newSateGandalf.Fello_ID = -1
    
    index_oldfello = currentState.pieces.index((-1,-1))
    updated_pieces = copy.copy(currentState.pieces)
    updated_pieces[index_oldfello] = (new_x,new_y)
    updated_pieces[0] = (new_x,new_y)
    updated_board = copy.copy(currentState.board)
    updated_board[new_x,new_y] = Cell(Fello_pos=True,Fello_ID=currentState.board[new_x,new_y].home_ID,home=True,home_ID=currentState.board[new_x,new_y].home_ID)
            
    return Board(currentState.rows,currentState.cols,newSateGandalf,updated_board, updated_pieces)
    
def move_to_fello_cell(new_x,new_y,curr_x,curr_y,currentState,move):
    newSateGandalf = copy.copy(currentState.gandalf)
    newSateGandalf.row, newSateGandalf.col = new_x,new_y
    newSateGandalf.path += move
    newSateGandalf.count = 0
    newSateGandalf.alone = False
    newSateGandalf.Fello_ID = currentState.board[new_x,new_y].Fello_ID
            
    updated_pieces = copy.copy(currentState.pieces)
    index_Fello = currentState.pieces.index((new_x,new_y))
    updated_pieces[index_Fello] = (-1,-1)
    updated_pieces[0] = (new_x,new_y)   
    updated_board = copy.copy(currentState.board)
    updated_board[new_x,new_y] = Cell(home=currentState.board[new_x,new_y].home,home_ID=currentState.board[new_x,new_y].home_ID)
            
    return Board(currentState.rows,currentState.cols,newSateGandalf,updated_board, updated_pieces)
  

def get_new_state(new_x,new_y,curr_x,curr_y,currentState,move):
    
    if(currentState.board[new_x,new_y].watched == True):
         return move_to_watched_cell(new_x,new_y,curr_x,curr_y,currentState,move)
        
    if(currentState.board[new_x,new_y].Fello_pos == True):
        if(currentState.gandalf.alone) and (currentState.board[new_x,new_y].Fello_ID != currentState.board[new_x,new_y].home_ID):# take this on
            return move_to_fello_cell(new_x,new_y,curr_x,curr_y,currentState,move)
        
        if(currentState.board[new_x,new_y].home == True):
            if(currentState.gandalf.alone == False and
                currentState.board[new_x,new_y].home_ID == currentState.gandalf.Fello_ID):#drop this one and take new one
                newSateGandalf = copy.copy(currentState.gandalf)
                newSateGandalf.row, newSateGandalf.col = new_x,new_y
                newSateGandalf.path += move
                newSateGandalf.count = 0
                newSateGandalf.alone = False
                newSateGandalf.Fello_ID = currentState.board[new_x,new_y].Fello_ID
          
                index_oldfello = currentState.pieces.index((-1,-1))
                index_newfello = currentState.pieces.index((new_x,new_y))
                updated_pieces = copy.copy(currentState.pieces)
                updated_pieces[index_newfello] = (-1,-1)
                updated_pieces[index_oldfello] = (new_x,new_y)
                updated_pieces[0] = (new_x,new_y)
                updated_board = copy.copy(currentState.board)
                updated_board[new_x,new_y] = Cell(Fello_pos=True,Fello_ID=currentState.board[new_x,new_y].home_ID,home=True,home_ID=currentState.board[new_x,new_y].home_ID)
            
                return Board(currentState.rows,currentState.cols,newSateGandalf,updated_board, updated_pieces)
    
    if(currentState.board[new_x,new_y].home == True):
        if(currentState.gandalf.alone == False and currentState.board[new_x,new_y].home_ID == currentState.gandalf.Fello_ID):
            return move_to_home_cell(new_x,new_y,curr_x,curr_y,currentState,move)

    return move_to_empty_cell(new_x,new_y,curr_x,curr_y,currentState,move)
            
        
            
def valid(curr_x,curr_y,new_x,new_y,currentState):
    if (new_x < 0 or new_y < 0 or new_x >= currentState.rows or new_y >= currentState.cols):
        return False
    
    if(currentState.board[new_x,new_y].Orc == True):
        return False
    
    if(currentState.board[new_x,new_y].watched == True):
        if(currentState.board[new_x,new_y].watched_orcID != currentState.board[curr_x,curr_y].watched_orcID):
            return True
        if(currentState.gandalf.count == currentState.board[new_x,new_y].watched_orcN):
            return False
        
    return True    


def manhattanDist(n,X1, Y1, X2, Y2):
    dist = Math.fabs(X2 - X1) + Math.fabs(Y2 - Y1)
    if(dist <= n):
        return True
    return False
    

    
def create_board(data):
    rows, cols = (data[0][0], data[0][1])
    initial_pieces = []
    grid = { (i,j):Cell() for i in range(rows) for j in range(cols) }
    orcs_count = data[3][0] 
    for i in range(0,orcs_count):
        x, y, n = data[4+i][0], data[4+i][1], data[4+i][2]
        r1 = x-n if x-n > 0 else 0
        r2 = x+n+1 if x+n+1 < cols else cols
        r3 = y-n if y-n >0 else 0
        r4 = y+n+1 if y+n+1 < rows else rows
        for j in range(r1, r2):
            for k in range(r3,r4):
                if(manhattanDist(n,x, y, j, k)):
                    grid[j,k].watched = True
                    grid[j,k].watched_orcN = n
                    grid[j,k].watched_orcID = i
                    
        grid[x,y].empty=False
        grid[x,y].Orc = True
    
    gandalf = Gandalf(data[1][0],data[1][1])
    initial_pieces.append((data[1][0],data[1][1]))
    grid[data[2][0],data[2][1]].castle = True
   
    Orks_count , Fellowships_count =data[3][0],data[3][1]
   
    for i in range(0, Fellowships_count):
        grid[data[4+Orks_count+i][0], data[4+Orks_count+i][1]].Fello_pos = True
        grid[data[4+Orks_count+i][0], data[4+Orks_count+i][1]].Fello_ID = i
        initial_pieces.append((data[4+Orks_count+i][0], data[4+Orks_count+i][1]))
        
    for i in range(0, Fellowships_count):
        grid[data[4+Orks_count+Fellowships_count+i][0], data[4+Orks_count+Fellowships_count+i][1]].home = True
        grid[data[4+Orks_count+Fellowships_count+i][0], data[4+Orks_count+Fellowships_count+i][1]].home_ID = i
       
    return Board(rows,cols,gandalf,grid,initial_pieces)


  
def show(board):
    arr = [['-' for x in range(board.cols)] for y in range(board.rows)] 
    for i in range(0,board.rows):
        for j in range(0,board.cols):
            if board.board[i,j].Orc == True:
                arr[i][j] = 'Or'
                continue
            if board.board[i,j].watched == True:
                arr[i][j] = 'w'
            if board.board[i,j].Fello_pos == True:
                arr[i][j] = 'F'+ str(board.board[i,j].Fello_ID)
            if board.board[i,j].home == True:
                arr[i][j] = 'H'+ str(board.board[i,j].home_ID)
            if board.board[i,j].Fello_pos == True and board.board[i,j].home == True:
                arr[i][j] = 'H'+ str(board.board[i,j].home_ID) + 'F'+ str(board.board[i,j].Fello_ID)
            if board.board[i,j].castle == True:
                arr[i][j] = 'D'
    x,y = board.gandalf.row, board.gandalf.col
    if arr[x][y]!='-':
        arr[x][y] += 'G'
    else:
        arr[x][y] = 'G'
    if(board.gandalf.Fello_ID != -1):
        arr[x][y] += 'F'+ str(board.gandalf.Fello_ID)  
   
    df = pd.DataFrame([[y for y in x] for x in arr])
    print(df,'\n\n')
       
def readFile(filename):
    with open(filename) as f:
        data = []
        for line in f: 
            data.append([int(x) for x in line.split()])
    return data



def Game(filename,repetition):
    total_time = 0
    for i in range(repetition):
        start_time = time.time()
        data = readFile(filename)
        initialState = create_board(data)
        numOfVisitedStates,finalState = BFSSearch(initialState)
        end_time = time.time()
        runtime = (end_time - start_time)
        total_time += runtime
        
   # show_path(finalState,initialState)
    std = (total_time*1000) / repetition
    path = 'None' if numOfVisitedStates == -1 else finalState.gandalf.path
    return std, path, numOfVisitedStates
    

if __name__ == "__main__":
    from memory_profiler import memory_usage
    file_names = ['test_00.txt', 'test_01.txt', 'test_02.txt', 'test_03.txt']
    repetition = 3
    std , path, numOfVisitedStates = [None]*len(file_names), [None]*len(file_names), [None]*len(file_names)

    print('BFS\n')
    for i in range(len(file_names)):
        std[i], path[i], numOfVisitedStates[i] = Game(file_names[i],repetition)
    for i in range(len(file_names)):
        print('\tTest:',file_names[i])
        print('\tExecution time =',std[i],'ms',
          '\n\tPath length =', len(path[i]),'\n\tPath =',path[i],'\n\tnumOfVisitedStates = ',numOfVisitedStates[i])
        print()
    
  


BFS

	Test: test_00.txt
	Execution time = 189.02738889058432 ms 
	Path length = 48 
	Path = RRRRRRRRRRRRRRRRRRRRRRRRRRRRRDURDURDURDURDRRRRRR 
	numOfVisitedStates =  7100

	Test: test_01.txt
	Execution time = 36.32020950317383 ms 
	Path length = 52 
	Path = DRRDRRDDDDDDLLLURUUUURRRRDLDLDDDLUURUURRRRDDDDLLLRRR 
	numOfVisitedStates =  1394

	Test: test_02.txt
	Execution time = 8.334080378214518 ms 
	Path length = 34 
	Path = RRRRRRRDDDDLLLLDLLLDDRUURURRRDRDDR 
	numOfVisitedStates =  404

	Test: test_03.txt
	Execution time = 82.99922943115234 ms 
	Path length = 66 
	Path = DDRDRDDDDDLLDRURUUUUUUUURRRRRRRLLDDDDDLLDDDDLLLLURUUUURRRRRRRDDDDD 
	numOfVisitedStates =  3963



# IDS

In [45]:
import numpy as np
import pandas as pd
import math as Math
import time
import itertools
import copy
import sys

maxLimit = 1e5

class Cell:
    def __init__(self,empty=True,castle=False,Orc=False,watched=False
               ,watched_orcN=-1,watched_orcID=-1,Fello_pos=False,Fello_ID=-1,home=False,home_ID=-1):
        self.empty = empty
        self.castle = castle
        self.Orc = Orc
        self.watched = watched
        self.watched_orcN = watched_orcN
        self.watched_orcID = watched_orcID
        self.Fello_pos = Fello_pos
        self.Fello_ID = Fello_ID
        self.home = home
        self.home_ID = home_ID
        
    def __eq__(self, other):
        contents1 = [self.__dict__[key] for key in self.__dict__.keys()]
        contents2 = [other.__dict__[key] for key in other.__dict__.keys()]
        if(contents1 == contents2):
            return True
        return False
    
class Gandalf:
    def __init__(self,row,col,path='',count = 0,alone = True, Fello_ID = -1):
        self.row = row
        self.col = col
        self.path = path
        self.count = count
        self.alone = alone
        self.Fello_ID = Fello_ID
    def __eq__(self, other):
        return self.count == other.count
        
class Board:
    def __init__(self,rows,cols,gandalf,board,pieces,depth = 0,parent=None):
        self.rows = rows
        self.cols = cols
        self.board = board
        self.gandalf = gandalf
        self.pieces = pieces
        self.depth = depth
        self.parent = parent
        
    def __eq__(self, other):
        return (self.pieces == other.pieces and self.depth <= other.depth and self.gandalf==other.gandalf)
 
    def __hash__(self):
        return hash((tuple(self.pieces)))

  
def show_path(finalState):
    
    if(finalState.depth == 0):
        show(finalState)
        return
    show_path(finalState.parent)
    show(finalState)
    
def checkEnd(state):
    
    if not state.board[(state.gandalf.row,state.gandalf.col)].castle:
        return False
        
    for i in range(1,len(state.pieces)):
        if(state.pieces[i]==(-1,-1)):
            return False
        if state.board[state.pieces[i]].home == True:
            if (state.board[state.pieces[i]].Fello_ID != state.board[state.pieces[i]].home_ID):
                    return False
        else:
            return False
    return True

numOfVisited = 0
def IDS(initialState):
  #  numOfVisited
    numOfVisited = 0
    for depth in range(0,sys.maxsize):
        if(depth > maxLimit):
            break;
        visited = set()
        result,numOfVisited, currentState = DFSWithDepth(initialState,depth,numOfVisited)
       # result, currentState =DFSRecrucive(initialState,depth,visited)
        if(result):
            return numOfVisited,currentState
        
    return -1,currentState




def DFSRecrucive(initialState,depth,visited):
    if(depth<0):
        return False,-1
    global numOfVisited
   
    if(checkEnd(initialState)):
        return True,initialState
   
   
    
    visited.add(initialState)
    numOfVisited += 1
    
    moves={'R':(0,1),'U':(-1,0),'D':(1,0), 'L':(0,-1)}
        
    x,y = initialState.gandalf.row, initialState.gandalf.col
    for move in moves: 
        if(valid(x,y,x+moves[move][0], y+moves[move][1],initialState) == False):
            continue
        newState = get_new_state(x+moves[move][0], y+moves[move][1],x,y,initialState,move)
        newState.depth = initialState.depth+1
        if  (newState not in visited):
            newState.parent = initialState 
            result, currentState = DFSRecrucive(newState,depth-1,visited)  
            if(result):
                return result, currentState
            
    return False,-1     
    
    
def DFSWithDepth(initialState,depth,numOfVisited):
  
    visited = set()
    frontier = []
    frontier_set = set()
    frontier.append(initialState)
    frontier_set.add(initialState)
    
    while len(frontier) != 0:
        currentState = frontier.pop()
        
        visited.add(currentState)
        numOfVisited += 1
        
        if(checkEnd(currentState)):
            return True,numOfVisited,currentState
 
        moves={'R':(0,1),'U':(-1,0),'D':(1,0), 'L':(0,-1)}
        
        x,y = currentState.gandalf.row, currentState.gandalf.col
       
        for move in moves: 
            if(currentState.depth+1 >  depth):
                break
            if(valid(x,y,x+moves[move][0], y+moves[move][1],currentState) == False):
                continue
            newState = get_new_state(x+moves[move][0], y+moves[move][1],x,y,currentState,move)
            newState.depth = currentState.depth+1
            if (newState not in visited) and (newState not in frontier_set):
                newState.parent = currentState
                frontier.append(newState)
                frontier_set.add(newState)        

    return False,-1,-1


def move_to_empty_cell(new_x,new_y,curr_x,curr_y,currentState,move):
    newSateGandalf = copy.copy(currentState.gandalf)
    newSateGandalf.row, newSateGandalf.col = new_x,new_y
    newSateGandalf.path += move
    newSateGandalf.count = 0
    updated_pieces = copy.copy(currentState.pieces)
    updated_pieces[0] = (new_x,new_y)
    newState = Board(currentState.rows,currentState.cols,newSateGandalf,currentState.board, updated_pieces)
    return newState

def move_to_watched_cell(new_x,new_y,curr_x,curr_y,currentState,move):
    newSateGandalf = copy.copy(currentState.gandalf)
    newSateGandalf.row, newSateGandalf.col = new_x,new_y
    newSateGandalf.path += move
    updated_pieces = copy.copy(currentState.pieces)
    updated_pieces[0] = (new_x,new_y)
    if(currentState.board[new_x,new_y].watched_orcID != currentState.board[curr_x,curr_y].watched_orcID):
        newSateGandalf.count = 1
        newState = Board(currentState.rows,currentState.cols,newSateGandalf,currentState.board, updated_pieces)
        return newState
    if(currentState.gandalf.count != currentState.board[new_x,new_y].watched_orcN):
        newSateGandalf.count += 1
        newState = Board(currentState.rows,currentState.cols,newSateGandalf,currentState.board, updated_pieces)
        return newState
       
def move_to_home_cell(new_x,new_y,curr_x,curr_y,currentState,move):
    newSateGandalf = copy.copy(currentState.gandalf)
    newSateGandalf.row, newSateGandalf.col = new_x,new_y
    newSateGandalf.path += move
    newSateGandalf.count = 0
    newSateGandalf.alone = True
    newSateGandalf.Fello_ID = -1
    
    index_oldfello = currentState.pieces.index((-1,-1))
    updated_pieces = copy.copy(currentState.pieces)
    updated_pieces[index_oldfello] = (new_x,new_y)
    updated_pieces[0] = (new_x,new_y)
    updated_board = copy.copy(currentState.board)
    updated_board[new_x,new_y] = Cell(Fello_pos=True,Fello_ID=currentState.board[new_x,new_y].home_ID,home=True,home_ID=currentState.board[new_x,new_y].home_ID)
            
    return Board(currentState.rows,currentState.cols,newSateGandalf,updated_board, updated_pieces)
    
def move_to_fello_cell(new_x,new_y,curr_x,curr_y,currentState,move):
    newSateGandalf = copy.copy(currentState.gandalf)
    newSateGandalf.row, newSateGandalf.col = new_x,new_y
    newSateGandalf.path += move
    newSateGandalf.count = 0
    newSateGandalf.alone = False
    newSateGandalf.Fello_ID = currentState.board[new_x,new_y].Fello_ID
            
    updated_pieces = copy.copy(currentState.pieces)
    index_Fello = currentState.pieces.index((new_x,new_y))
    updated_pieces[index_Fello] = (-1,-1)
    updated_pieces[0] = (new_x,new_y)   
    updated_board = copy.copy(currentState.board)
    updated_board[new_x,new_y] = Cell(home=currentState.board[new_x,new_y].home,home_ID=currentState.board[new_x,new_y].home_ID)
            
    return Board(currentState.rows,currentState.cols,newSateGandalf,updated_board, updated_pieces)
  

def get_new_state(new_x,new_y,curr_x,curr_y,currentState,move):
    
    if(currentState.board[new_x,new_y].watched == True):
         return move_to_watched_cell(new_x,new_y,curr_x,curr_y,currentState,move)
        
    if(currentState.board[new_x,new_y].Fello_pos == True):
        if(currentState.gandalf.alone) and (currentState.board[new_x,new_y].Fello_ID != currentState.board[new_x,new_y].home_ID):# take this on
            return move_to_fello_cell(new_x,new_y,curr_x,curr_y,currentState,move)
        
        if(currentState.board[new_x,new_y].home == True):
            if(currentState.gandalf.alone == False and
                currentState.board[new_x,new_y].home_ID == currentState.gandalf.Fello_ID):#drop this one and take new one
                newSateGandalf = copy.copy(currentState.gandalf)
                newSateGandalf.row, newSateGandalf.col = new_x,new_y
                newSateGandalf.path += move
                newSateGandalf.count = 0
                newSateGandalf.alone = False
                newSateGandalf.Fello_ID = currentState.board[new_x,new_y].Fello_ID
          
                index_oldfello = currentState.pieces.index((-1,-1))
                index_newfello = currentState.pieces.index((new_x,new_y))
                updated_pieces = copy.copy(currentState.pieces)
                updated_pieces[index_newfello] = (-1,-1)
                updated_pieces[index_oldfello] = (new_x,new_y)
                updated_pieces[0] = (new_x,new_y)
                updated_board = copy.copy(currentState.board)
                updated_board[new_x,new_y] = Cell(Fello_pos=True,Fello_ID=currentState.board[new_x,new_y].home_ID,home=True,home_ID=currentState.board[new_x,new_y].home_ID)
            
                return Board(currentState.rows,currentState.cols,newSateGandalf,updated_board, updated_pieces)
    
    if(currentState.board[new_x,new_y].home == True):
        if(currentState.gandalf.alone == False and currentState.board[new_x,new_y].home_ID == currentState.gandalf.Fello_ID):
            return move_to_home_cell(new_x,new_y,curr_x,curr_y,currentState,move)

    return move_to_empty_cell(new_x,new_y,curr_x,curr_y,currentState,move)
            
        
        
def valid(curr_x,curr_y,new_x,new_y,currentState):
    if (new_x < 0 or new_y < 0 or new_x >= currentState.rows or new_y >= currentState.cols):
       
        return False
    if(currentState.board[new_x,new_y].Orc == True):
        return False
    
    if(currentState.board[new_x,new_y].watched == True):
        if(currentState.board[new_x,new_y].watched_orcID != currentState.board[curr_x,curr_y].watched_orcID):
            return True
        if(currentState.gandalf.count == currentState.board[new_x,new_y].watched_orcN):
            return False
        
    return True    
    
def manhattanDist(n,X1, Y1, X2, Y2):
    dist = Math.fabs(X2 - X1) + Math.fabs(Y2 - Y1)
    if(dist <= n):
        return True
    return False
    

    
def create_board(data):
    rows, cols = (data[0][0], data[0][1])
    initial_pieces = []
    grid = { (i,j):Cell() for i in range(rows) for j in range(cols) }
    orcs_count = data[3][0] 
    for i in range(0,orcs_count):
        x, y, n = data[4+i][0], data[4+i][1], data[4+i][2]
        r1 = x-n if x-n > 0 else 0
        r2 = x+n+1 if x+n+1 < cols else cols
        r3 = y-n if y-n >0 else 0
        r4 = y+n+1 if y+n+1 < rows else rows
        for j in range(r1, r2):
            for k in range(r3,r4):
                if(manhattanDist(n,x, y, j, k)):
                    grid[j,k].watched = True
                    grid[j,k].watched_orcN = n
                    grid[j,k].watched_orcID = i
                    
        grid[x,y].empty=False
        grid[x,y].Orc = True
    
    gandalf = Gandalf(data[1][0],data[1][1])
    initial_pieces.append((data[1][0],data[1][1]))
    grid[data[2][0],data[2][1]].castle = True
    Orks_count , Fellowships_count =data[3][0],data[3][1]
    
    Fellowships_pos = {}
    Fellowships_dest = {}
    for i in range(0, Fellowships_count):
        grid[data[4+Orks_count+i][0], data[4+Orks_count+i][1]].Fello_pos = True
        grid[data[4+Orks_count+i][0], data[4+Orks_count+i][1]].Fello_ID = i
        initial_pieces.append((data[4+Orks_count+i][0], data[4+Orks_count+i][1]))
        
    for i in range(0, Fellowships_count):
        grid[data[4+Orks_count+Fellowships_count+i][0], data[4+Orks_count+Fellowships_count+i][1]].home = True
        grid[data[4+Orks_count+Fellowships_count+i][0], data[4+Orks_count+Fellowships_count+i][1]].home_ID = i
       
    return Board(rows,cols,gandalf,grid,initial_pieces)


def show(board):
    arr = [['-' for x in range(board.cols)] for y in range(board.rows)] 
    for i in range(0,board.rows):
        for j in range(0,board.cols):
            if board.board[i,j].Orc == True:
                arr[i][j] = 'Or'
                continue
            if board.board[i,j].watched == True:
                arr[i][j] = 'w'
            if board.board[i,j].Fello_pos == True:
                arr[i][j] = 'F'+ str(board.board[i,j].Fello_ID)
            if board.board[i,j].home == True:
                arr[i][j] = 'H'+ str(board.board[i,j].home_ID)
            if board.board[i,j].Fello_pos == True and board.board[i,j].home == True:
                arr[i][j] = 'H'+ str(board.board[i,j].home_ID) + 'F'+ str(board.board[i,j].Fello_ID)
            if board.board[i,j].castle == True:
                arr[i][j] = 'D'
    x,y = board.gandalf.row, board.gandalf.col
    if arr[x][y]!='-':
        arr[x][y] += 'G'
    else:
        arr[x][y] = 'G'
    if(board.gandalf.Fello_ID != -1):
        arr[x][y] += 'F'+ str(board.gandalf.Fello_ID)  
   
    df = pd.DataFrame([[y for y in x] for x in arr])
    print(df,'\n\n')
        
def readFile(filename):
    with open(filename) as f:
        data = []
        for line in f: 
            data.append([int(x) for x in line.split()])
    return data
     
def Game(filename,repetition):

    total_time = 0
    for i in range(repetition):
        start_time = time.time()
        data = readFile(filename)
        initialState = create_board(data)
        numOfVisitedStates,finalState1 = IDS(initialState)
        
        end_time = time.time()
        runtime = (end_time - start_time)
        total_time += runtime
  
            
    std = (total_time) / repetition 
    path = 'None' if numOfVisitedStates == -1 else finalState1.gandalf.path
    return std, path, numOfVisitedStates
    
if __name__ == "__main__":
    file_names = ['test_00.txt', 'test_01.txt', 'test_02.txt' ,'test_03.txt']
  
    std , path, numOfVisitedStates = [None]*len(file_names), [None]*len(file_names), [None]*len(file_names)
    
    repetition = 3
    for i in range(len(file_names)):
        std[i], path[i], numOfVisitedStates[i] = Game(file_names[i],repetition)
    for i in range(len(file_names)):
        print('\tTest:',file_names[i])
        print('\tExecution time =',std[i],'s',
          '\n\tPath length =', len(path[i]),'\n\tPath =',path[i],'\n\tnumOfVisitedStates = ',numOfVisitedStates[i])
        print()

	Test: test_00.txt
	Execution time = 3.194553772608439 s 
	Path length = 48 
	Path = RRRRRRRRRRRRRRRRRRRRRRRRRRRRRDURDURDURDURDRRRRRR 
	numOfVisitedStates =  21506

	Test: test_01.txt
	Execution time = 5.260388692220052 s 
	Path length = 52 
	Path = DRRDRRLLDDDDDLDURUUUURRRRLLDDDLDDUURUURRRRDDDLDLLRRR 
	numOfVisitedStates =  12616

	Test: test_02.txt
	Execution time = 0.3450593948364258 s 
	Path length = 34 
	Path = RRRRRRRLLLLDDDDLDLLDDRUURURRRDRDDR 
	numOfVisitedStates =  1529

	Test: test_03.txt
	Execution time = 20.390227953592937 s 
	Path length = 66 
	Path = DDRDRDDLDLDDDRURUUUUUUUURRRRRRRLLLLLLLDDDDDDDDLDURUUUURRRRRRRDDDDD 
	numOfVisitedStates =  11336



# AStar

In [47]:
import numpy as np
import pandas as pd
import math as Math
import time
import itertools
import copy
import sys
from heapq import heapify, heappush, heappop

        
class Cell:
    def __init__(self,empty=True,castle=False,Orc=False,watched=False
               ,watched_orcN=-1,watched_orcID=-1,Fello_pos=False,Fello_ID=-1,home=False,home_ID=-1):
        self.empty = empty
        self.castle = castle
        self.Orc = Orc
        self.watched = watched
        self.watched_orcN = watched_orcN
        self.watched_orcID = watched_orcID
        self.Fello_pos = Fello_pos
        self.Fello_ID = Fello_ID
        self.home = home
        self.home_ID = home_ID
        
    def __eq__(self, other):
        contents1 = [self.__dict__[key] for key in self.__dict__.keys()]
        contents2 = [other.__dict__[key] for key in other.__dict__.keys()]
        if(contents1 == contents2):
            return True
        return False
    
class Gandalf:
    def __init__(self,row,col,path='',count = 0,alone = True, Fello_ID = -1):
        self.row = row
        self.col = col
        self.path = path
        self.count = count
        self.alone = alone
        self.Fello_ID = Fello_ID
    def __eq__(self, other):
        return (self.row == other.row and self.col == other.col and self.col == other.col
            and self.count == other.count and self.path == other.path)
        
class Board:
    def __init__(self,rows,cols,gandalf,board,pieces,f=sys.maxsize,g=sys.maxsize,h=0, home_positions=[],parent=None):
        self.rows = rows
        self.cols = cols
        self.board = board
        self.gandalf = gandalf
        self.pieces = pieces
        self.g = g
        self.f = f
        self.h = h
        self.home_positions =  home_positions
        self.parent = parent
   
    def __eq__(self, other):
        if (self.pieces == other.pieces and self.gandalf == other.gandalf):
            return True
        return False
       
    def __hash__(self):
        return hash((tuple(self.pieces)))
    def __lt__(self, other):
        return self.f < other.f

def show_path(finalState):
    
    if(finalState.g == 0):
        show(finalState)
        return
    construct_path(finalState.parent)
    show(finalState)
       
    
def checkEnd(state):
    
    if not state.board[(state.gandalf.row,state.gandalf.col)].castle:
        return False
        
    for i in range(1,len(state.pieces)):
        if(state.pieces[i]==(-1,-1)):
            return False
        if state.board[state.pieces[i]].home == True:
            if (state.board[state.pieces[i]].Fello_ID != state.board[state.pieces[i]].home_ID):
                    return False
        else:
            return False
    return True


def ManhattanDist(X1, Y1, X2, Y2):
    dist = Math.fabs(X2 - X1) + Math.fabs(Y2 - Y1)
    return dist


def calculate_heuristic(state):
    
    total_dist = 0
    for i in range (0,len(state.home_positions)):
        if(state.pieces[i] == (-1,-1)):
            total_dist += ManhattanDist(state.pieces[0][0], state.pieces[0][1], state.home_positions[i][0], state.home_positions[i][1])
        else:
            total_dist += ManhattanDist(state.pieces[i][0], state.pieces[i][1], state.home_positions[i][0], state.home_positions[i][1])
    return total_dist     
 

def A_Star(initialState,weight):
  
    numOfVisitedStates = 0
   
    visited_set = {}
    frontier = []
    visited = []
    
    heapify(frontier)
    initialState.g = 0
    initialState.h = weight*calculate_heuristic(initialState)
    initialState.f = initialState.h
   
    heappush(frontier, initialState)
    visited.append(initialState)
    visited_set[tuple(initialState.pieces)]=len(visited)-1
    
    while len(frontier) != 0:
       
        currentState = heappop(frontier)
        numOfVisitedStates+=1
        
        if(checkEnd(currentState)): 
            return numOfVisitedStates,currentState
        
        moves={'R':(0,1),'U':(-1,0),'D':(1,0), 'L':(0,-1)}
       
        gandalfRow, gandalfCol = currentState.gandalf.row, currentState.gandalf.col
        for move in moves: 
            
            
            newRow,newCol = gandalfRow+moves[move][0], gandalfCol+moves[move][1]
            if(valid(gandalfRow, gandalfCol, newRow,newCol, currentState) == False):
                continue
            newState = get_new_state(newRow,newCol,gandalfRow, gandalfCol, currentState, move)
            newState.parent = currentState
            newh = weight*calculate_heuristic(newState)
            stateIndex = visited_set.get(tuple(newState.pieces))
           
            curr_g =0
            if stateIndex != None:
                curr_g = visited[stateIndex].g
            else:
                
                newState.g = currentState.g + 1
                newState.f = newh + newState.g
                newState.h = newh
                heappush(frontier, newState)
                visited.append(newState)
                visited_set[tuple(newState.pieces)]=len(visited)-1
                continue
           
            new_g = currentState.g + 1
            if(new_g  < curr_g):
                visited[stateIndex].parent = currentState
                visited[stateIndex].g = new_g
                visited[stateIndex].f = newh + new_g
                visited[stateIndex].h = newh
                if(newState not in frontier):
                    newState.g = new_g
                    newState.f = newh + new_g
                    newState.h = newh
                    heappush(frontier, newState)
                    visited.append(newState)
                    visited_set[tuple(newState.pieces)]=len(visited)-1  
                 
    return -1,False


def get_new_state(new_x,new_y,curr_x,curr_y,currentState,move):
    
    if(currentState.board[new_x,new_y].watched == True):
        
        if(currentState.board[new_x,new_y].watched_orcID != currentState.board[curr_x,curr_y].watched_orcID):
            new_gan = Gandalf(new_x,new_y,currentState.gandalf.path+move,1,currentState.gandalf.alone,currentState.gandalf.Fello_ID)
            index = currentState.pieces.index((curr_x,curr_y))
            updated_pieces = copy.copy(currentState.pieces)
            updated_pieces[index] = (new_x,new_y)
            return Board(currentState.rows,currentState.cols,new_gan,currentState.board, updated_pieces,home_positions=currentState.home_positions)
        
        if(currentState.gandalf.count != currentState.board[new_x,new_y].watched_orcN):
            new_gan = Gandalf(new_x,new_y,currentState.gandalf.path+move,currentState.gandalf.count+1,currentState.gandalf.alone,currentState.gandalf.Fello_ID)
            index = currentState.pieces.index((curr_x,curr_y))
            updated_pieces = copy.copy(currentState.pieces)
            updated_pieces[index] = (new_x,new_y)
            return Board(currentState.rows,currentState.cols,new_gan,currentState.board, updated_pieces,home_positions=currentState.home_positions)
        
    if(currentState.board[new_x,new_y].Fello_pos == True):
        if(currentState.gandalf.alone) and (currentState.board[new_x,new_y].Fello_ID != currentState.board[new_x,new_y].home_ID):# take this on
            new_gan = Gandalf(new_x,new_y,currentState.gandalf.path+move,0,False,currentState.board[new_x,new_y].Fello_ID)
            index_gan= currentState.pieces.index((curr_x,curr_y))
            updated_pieces = copy.copy(currentState.pieces)
            updated_pieces[index_gan] = (new_x,new_y)
            index_Fello = currentState.pieces.index((new_x,new_y))
            updated_pieces[index_Fello] = (-1,-1)
            updated_board = copy.copy(currentState.board)
            updated_board[new_x,new_y] = Cell(home=currentState.board[new_x,new_y].home,home_ID=currentState.board[new_x,new_y].home_ID)
            return Board(currentState.rows,currentState.cols,new_gan,updated_board, updated_pieces,home_positions=currentState.home_positions)

        if(currentState.board[new_x,new_y].home == True):
            if(currentState.gandalf.alone == False and
                currentState.board[new_x,new_y].home_ID == currentState.gandalf.Fello_ID):#drop this one and take new one

                new_gan = Gandalf(new_x,new_y,currentState.gandalf.path+move,0,False,currentState.board[new_x,new_y].Fello_ID)
                index_oldfello = currentState.pieces.index((-1,-1))
                index_newfello = currentState.pieces.index((new_x,new_y))
                index_gan= currentState.pieces.index((curr_x,curr_y))
                updated_pieces = copy.copy(currentState.pieces)
                updated_pieces[index_gan] = (new_x,new_y)
                updated_pieces[index_newfello] = (-1,-1)
                updated_pieces[index_oldfello] = (new_x,new_y)
                updated_board = copy.copy(currentState.board)
                updated_board[new_x,new_y] = Cell(Fello_pos=True,Fello_ID=currentState.board[new_x,new_y].home_ID,home=True,home_ID=currentState.board[new_x,new_y].home_ID)
                return Board(currentState.rows,currentState.cols,new_gan,updated_board, updated_pieces,home_positions=currentState.home_positions)
     
    
    if(currentState.board[new_x,new_y].home == True):
        if(currentState.gandalf.alone == False and currentState.board[new_x,new_y].home_ID == currentState.gandalf.Fello_ID):

            new_gan = Gandalf(new_x,new_y,currentState.gandalf.path+move,0,True,-1)
            index_oldfello = currentState.pieces.index((-1,-1))
            index_gan= currentState.pieces.index((curr_x,curr_y))
            updated_pieces = copy.copy(currentState.pieces)
            updated_pieces[index_gan] = (new_x,new_y)
            updated_pieces[index_oldfello] = (new_x,new_y)
            updated_board = copy.copy(currentState.board)
            updated_board[new_x,new_y] = Cell(Fello_pos=True,Fello_ID=currentState.board[new_x,new_y].home_ID,home=True,home_ID=currentState.board[new_x,new_y].home_ID)
            return Board(currentState.rows,currentState.cols,new_gan,updated_board, updated_pieces,home_positions=currentState.home_positions)            
       
    new_gan = Gandalf(new_x,new_y,currentState.gandalf.path+move,0,currentState.gandalf.alone,currentState.gandalf.Fello_ID)
    index = currentState.pieces.index((curr_x,curr_y))
    updated_pieces = copy.copy(currentState.pieces)
    updated_pieces[index] = (new_x,new_y)
    return Board(currentState.rows,currentState.cols,new_gan,currentState.board, updated_pieces,home_positions=currentState.home_positions)
        

        
def valid(curr_x,curr_y,new_x,new_y,currentState):
    
    if (new_x < 0 or new_y < 0 or new_x >= currentState.rows or new_y >= currentState.cols):
        return False
    
    if(currentState.board[new_x,new_y].Orc == True):
        return False
    
    if(currentState.board[new_x,new_y].watched == True):
        if(currentState.board[new_x,new_y].watched_orcID != currentState.board[curr_x,curr_y].watched_orcID):
            return True
        if(currentState.gandalf.count == currentState.board[new_x,new_y].watched_orcN):
            return False
        
    return True    
    
    
def create_board(data):
    rows, cols = (data[0][0], data[0][1])
    initial_pieces = []
    home_positions = []
    grid = { (i,j):Cell() for i in range(rows) for j in range(cols) }
    orcs_count = data[3][0] 
    for i in range(0,orcs_count):
        x, y, n = data[4+i][0], data[4+i][1], data[4+i][2]
        r1 = x-n if x-n > 0 else 0
        r2 = x+n+1 if x+n+1 < cols else cols
        r3 = y-n if y-n >0 else 0
        r4 = y+n+1 if y+n+1 < rows else rows
        for j in range(r1, r2):
            for k in range(r3,r4):
                if(ManhattanDist(x, y, j, k) <= n):
                    grid[j,k].watched = True
                    grid[j,k].watched_orcN = n
                    grid[j,k].watched_orcID = i
                    
        grid[x,y].empty=False
        grid[x,y].Orc = True
    
    gandalf = Gandalf(data[1][0],data[1][1])
    initial_pieces.append((data[1][0],data[1][1]))
   
    grid[data[2][0],data[2][1]].castle = True
    home_positions.append((data[2][0],data[2][1]))
   
    Orcs_count , Fellowships_count =data[3][0],data[3][1]
    
    for i in range(0, Fellowships_count):
        grid[data[4+Orcs_count+i][0], data[4+Orcs_count+i][1]].Fello_pos = True
        grid[data[4+Orcs_count+i][0], data[4+Orcs_count+i][1]].Fello_ID = i
        initial_pieces.append((data[4+Orcs_count+i][0], data[4+Orcs_count+i][1]))
        grid[data[4+Orcs_count+Fellowships_count+i][0], data[4+Orcs_count+Fellowships_count+i][1]].home = True
        grid[data[4+Orcs_count+Fellowships_count+i][0], data[4+Orcs_count+Fellowships_count+i][1]].home_ID = i
        home_positions.append((data[4+Orcs_count+Fellowships_count+i][0], data[4+Orcs_count+Fellowships_count+i][1]))
    
    return Board(rows,cols,gandalf,grid,initial_pieces, home_positions=home_positions)
    

def show(board):
    arr = [['-' for x in range(board.cols)] for y in range(board.rows)] 
    for i in range(0,board.rows):
        for j in range(0,board.cols):
            if board.board[i,j].Orc == True:
                arr[i][j] = 'Or'
                continue
            if board.board[i,j].watched == True:
                arr[i][j] = 'w'
            if board.board[i,j].Fello_pos == True:
                arr[i][j] = 'F'+ str(board.board[i,j].Fello_ID)
            if board.board[i,j].home == True:
                arr[i][j] = 'H'+ str(board.board[i,j].home_ID)
            if board.board[i,j].Fello_pos == True and board.board[i,j].home == True:
                arr[i][j] = 'H'+ str(board.board[i,j].home_ID) + 'F'+ str(board.board[i,j].Fello_ID)
            if board.board[i,j].castle == True:
                arr[i][j] = 'D'
    x,y = board.gandalf.row, board.gandalf.col
    if arr[x][y]!='-':
        arr[x][y] += 'G'
    else:
        arr[x][y] = 'G'
    if(board.gandalf.Fello_ID != -1):
        arr[x][y] += 'F'+ str(board.gandalf.Fello_ID)  
   
    df = pd.DataFrame([[y for y in x] for x in arr])
    print(df,'\n\n')
        
        
def readFile(filename):
    with open(filename) as f:
        data = []
        for line in f: 
            data.append([int(x) for x in line.split()])
    return data

def Game(filename,repetition,weight):
    total_time = 0
    for i in range(repetition):
        start_time = time.time()
        data = readFile(filename)
        initialState = create_board(data)
        numOfVisitedStates,finalState = A_Star(initialState,weight)
        end_time = time.time()
        runtime = (end_time - start_time)
        total_time += runtime
   # show_path(finalState)
 
    std = (total_time*1000) / repetition     
    path = 'None' if numOfVisitedStates == -1 else finalState.gandalf.path
    return std, path, numOfVisitedStates
  
    
class AStar:
    def __init__(self,file_names,repetition=1):
        self.fileNames = file_names
        self.repetition = repetition
    def Consistent(self):
        Setup(self.fileNames,self.repetition, 1,'Consistent')
    def Weighted(self,weight):
        Setup(self.fileNames,self.repetition,weight,'Weighted with weight '+str(weight))
        
def Setup(file_names,repetition,weight, Algorithm):
    
    std , path, numOfVisitedStates = [None]*len(file_names), [None]*len(file_names), [None]*len(file_names)
   
    for i in range(len(file_names)):
        std[i], path[i], numOfVisitedStates[i] = Game(file_names[i],repetition,weight)
        
    print('Algorithm:',Algorithm,'\n')
    for i in range(len(file_names)):
        print('\tTest:',file_names[i])
        print('\tExecution time =',std[i],'ms',
          '\n\tPath length =', len(path[i]),'\n\tPath =',path[i],'\n\tnumOfVisitedStates = ',numOfVisitedStates[i])
        print()
    print()
    
if __name__ == "__main__":
    
    file_names = ['test_00.txt', 'test_01.txt', 'test_02.txt', 'test_03.txt']
    
    Astar = AStar(file_names,3)
    Astar.Consistent()
    Astar.Weighted(1.5)
    Astar.Weighted(3)
    

Algorithm: Consistent 

	Test: test_00.txt
	Execution time = 33.643086751302086 ms 
	Path length = 48 
	Path = RRRRRRRRRRRRRRRRRRRRRRRRRRRRRDRUDRUDRUDRUDRRRRRR 
	numOfVisitedStates =  767

	Test: test_01.txt
	Execution time = 23.999929428100586 ms 
	Path length = 52 
	Path = RRRRDDDDDDDDLLLRRRUUURURUDLDLDDDLRRRRRUUUUDDDDLLLRRR 
	numOfVisitedStates =  795

	Test: test_02.txt
	Execution time = 4.666725794474284 ms 
	Path length = 34 
	Path = RRRRRRRDDDDLLLLDLLLRDDRUURURRRDRDD 
	numOfVisitedStates =  176

	Test: test_03.txt
	Execution time = 35.67639986673991 ms 
	Path length = 66 
	Path = RRDDDDDDDDLLRDRRRRRRRRUUUUUUUUULLDDDDDLLDDDLDLLLRRRRUUUUURRRRDDDDD 
	numOfVisitedStates =  1072


Algorithm: Weighted with weight 1.5 

	Test: test_00.txt
	Execution time = 23.999691009521484 ms 
	Path length = 48 
	Path = RRRRRRRRRRRRRRRRRRRRRRRRRRRRRDRUDRUDRUDRUDRRRRRR 
	numOfVisitedStates =  565

	Test: test_01.txt
	Execution time = 20.66644032796224 ms 
	Path length = 54 
	Path = RRRRRDDLDDDDDDLLLRR